In [46]:
import pandas as pd
import os, sys

sys.path = list(set(sys.path + [ '../case_study_word_distribution_trends/iteration_01' ]))

import utility
import text_corpus
import corpus_vectorizer
import types


In [47]:
# df = pd.read_excel('./data/year+text_window.xlsx')
# df.to_csv('./data/year+text_window.txt', sep='\t')

df = pd.read_csv('./data/year+text_window.txt', sep='\t')[['year', 'txt']]
df.head()

,year,txt
0,1945,andre red f l lönnegren ( ansvarig utgivare po...
1,1945,åter blivit en internationell maktfaktor en se...
2,1945,av en tysk ockupation för att trygga sin polit...
3,1945,om vad som inträffat i för oss främmande polit...
4,1945,har kommit som det nya årets första stora poli...


In [80]:
df[df.txt.isna()]

,year,txt
63,1945,NaN
123,1945,NaN
521,1945,NaN
706,1945,NaN
1727,1945,NaN
...,...,...
929249,1989,NaN
929398,1989,NaN
929732,1989,NaN
929748,1989,NaN


In [96]:
class DataFrameReader:

    def __init__(self, df, year=None):
        
        self.df = df
        
        if year is not None:
            self.df = self.df[self.df.year == year]
                              
        if len(self.df[self.df.txt.isna()]) > 0:
            print('Warn: {} n/a rows encountered'.format(len(self.df[self.df.txt.isna()])))
            self.df = self.df.dropna()
            
        self.iterator = None
        self.metadata = [ types.SimpleNamespace(filename=str(i), year=r) for i, r in enumerate(self.df.year.values)]
        self.metadict = { x.filename: x for x in (self.metadata or [])}
        self.filenames = [ x.filename for x in self.metadata ]
        
    def __iter__(self):
                              
        self.iterator = None
        return self

    def __next__(self):
                              
        if self.iterator is None:
            self.iterator = self.get_iterator()
                              
        return next(self.iterator)

    def get_iterator(self):
        return ((str(i), x) for i,x in enumerate(self.df.txt))


SyntaxError: invalid syntax (<ipython-input-96-e41147caca97>, line 10)

In [94]:
reader = DataFrameReader(df, 1967)

tokenizer = lambda x: x.split()

corpus = text_corpus.ProcessedCorpus(reader, tokenizer=tokenizer, min_len=1, isalnum=False, to_lower=False, numerals=False)
vectorizer = corpus_vectorizer.CorpusVectorizer() 
vectorizer.fit_transform(corpus)

len(vectorizer.vocabulary.keys())



Warn: 137 n/a rows encountered


36868

In [95]:
vectorizer.X.shape

(23513, 36868)